## RAG SQL Agent

The idea is to now be able to join the capabilities of rag and the sql agent in one query. We first need to pass our dummy data into a database (we will look how to automatize this later)

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

## Dummy database and document creation

### Database

In [8]:
import pandas as pd
import sqlite3

# Define your Excel files and their corresponding table names in the database
files_and_tables = {
    '../data/dummy_companies/digital_marketing_agency/database/Content_Calendar_Tech_Innovations_Ltd.xlsx': 'content_calendar',
    '../data/dummy_companies/digital_marketing_agency/database/Digital_Marketing_Campaigns.xlsx': 'digital_marketing_campaigns',
    '../data/dummy_companies/digital_marketing_agency/database/Performance_Metrics_Table.xlsx': 'performance_metrics',
    '../data/dummy_companies/digital_marketing_agency/database/Client_Info.xlsx': 'client_info'
}

# Connect to SQLite database (this will create the file if it doesn't exist)
conn = sqlite3.connect('../db/digital_marketing_agency.db')

# Loop through each file and table name, load the data and write it to the database
for file_path, table_name in files_and_tables.items():
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(file_path)
    
    # Write the DataFrame to a table in the SQLite database
    df.to_sql(table_name, conn, if_exists='replace', index=False)  # 'replace' to overwrite existing tables

# Close the connection to the database
conn.close()

print("Database has been created and populated with Excel data.")


Database has been created and populated with Excel data.


### Document

In [36]:
from docx import Document
import os

def get_docx_text(path):
    """
    Extract text from a .docx file.
    """
    doc = Document(path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

def read_documents_from_folder(folder_path):
    """
    Read all .docx files in the specified folder and return their contents in a list.
    """
    document_folder_path = os.path.join(folder_path, 'documents')
    documents_content = []
    for filename in os.listdir(document_folder_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(document_folder_path, filename)
            document_text = get_docx_text(file_path)
            documents_content.append(document_text)
    return documents_content

# Specify the path to your "documents" folder
folder_path = '../data/dummy_companies/digital_marketing_agency'

# Read the content of all documents in the folder
documents_list = read_documents_from_folder(folder_path)

# Now `documents_list` contains the content of each document as a string
print(documents_list)


["Budget and Billing Records: Tech Innovations Ltd.\nIntroduction\nThis document outlines the budget allocation, spending, and billing records for Tech Innovations Ltd. related to their digital marketing campaigns managed by the Digital Marketing Agency. It provides a detailed breakdown of costs associated with each campaign, payments made, and any adjustments based on campaign performance.\nBudget Overview\nThe total marketing budget allocated for the year 2024 was $50,000, distributed across various campaigns to support the strategic objectives of Tech Innovations Ltd. The budget was allocated as follows:\n- Tech Trendsetters Campaign: $10,000\n- Product Launch Campaign: $15,000\n- Continuous Online Presence: $10,000\n- Special Projects & Innovations: $15,000\nSpending Summary\nAs of the end of Q2 2024, the spending on marketing campaigns is as follows:\n- Tech Trendsetters Campaign: $9,500 (Completed)\n- Product Launch Campaign: $7,000 (Ongoing)\n- Continuous Online Presence: $5,500

In [37]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

docsearch = FAISS.from_texts(documents_list, embeddings)


In [83]:
docsearch.index.ntotal

5

In [24]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 2})




In [8]:
retrieved_docs = retriever.invoke("How has the campaign been progressing?")



In [54]:
retrieved_docs

[Document(page_content="Campaign Performance Report: Tech Trendsetters\n1. Campaign Overview\nThe 'Tech Trendsetters' campaign was designed to increase brand awareness and showcase Tech Innovations Ltd. as a leader in cutting-edge technology solutions. The campaign leveraged multiple digital channels, including social media, email marketing, and PPC advertising, to reach a broad audience of tech enthusiasts and professionals.\n2. Objectives\nThe main objectives of the 'Tech Trendsetters' campaign were to:\n- Increase website traffic by 20%.\n- Grow social media engagement by 30%.\n- Generate 500 new leads for the sales team.\n- Enhance brand recognition within the technology industry.\n3. Performance Metrics\nThe campaign 'Tech Trendsetters' achieved the following key performance metrics over its 3-month duration:\n- Website Traffic: 25% increase\n- Social Media Engagement: 35% increase\n- New Leads Generated: 600\n- PPC Campaign ROI: 120%\n4. Analysis\nThe 'Tech Trendsetters' campaign

## Loading the database

In [3]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI


db = SQLDatabase.from_uri("sqlite:///../db/digital_marketing_agency.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM content_calendar LIMIT 10;")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0))
context = toolkit.get_context()
tools = toolkit.get_tools()

sqlite
['client_info', 'content_calendar', 'digital_marketing_campaigns', 'performance_metrics']


"[('2024-01-15', 'Blog Post', 'The Future of AI in Business', 'Alice M.', 'Scheduled'), ('2024-01-22', 'Social Media Post', 'Upcoming Tech Trends - 2024', 'Team', 'Scheduled'), ('2024-02-05', 'Email Newsletter', 'Exclusive Offer: AI Tools for Your Business', 'Marketing Dept.', 'Draft'), ('2024-02-19', 'Blog Post', 'How Blockchain is Changing Finance', 'Bob C.', 'Scheduled'), ('2024-03-05', 'Social Media Post', 'Meet Our Team!', 'Team', 'Scheduled'), ('2024-03-19', 'Email Newsletter', 'Spring Product Launch - Sneak Peek!', 'Marketing Dept.', 'Draft'), ('2024-04-02', 'Blog Post', 'Improving Your Cybersecurity', 'Charlie D.', 'Planned'), ('2024-04-16', 'Social Media Post', 'Productivity Hacks: Tech Edition', 'Team', 'Planned'), ('2024-04-30', 'Blog Post', 'Why Cloud Storage is Essential', 'Diana E.', 'Planned'), ('2024-05-14', 'Email Newsletter', 'Major Update Announcement!', 'Marketing Dept.', 'Planned')]"

## Agent

In [28]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [7]:
agent_executor.invoke(
    "What type of service do we provide to Tech Innovations?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


client_info, content_calendar, digital_marketing_campaigns, performance_metrics
Invoking: `sql_db_schema` with `{'table_names': 'client_info'}`



CREATE TABLE client_info (
	"ClientID" INTEGER, 
	"CompanyName" TEXT, 
	"Industry" TEXT, 
	"ContactName" TEXT, 
	"ContactEmail" TEXT, 
	"ServicePackage" TEXT
)

/*
3 rows from client_info table:
ClientID	CompanyName	Industry	ContactName	ContactEmail	ServicePackage
1	Tech Innovations Ltd.	Technology	Alex Johnson	alex.j@techinnovations.com	Premium
2	Green Earth Foods	Food & Beverage	Jordan Lee	jordan.l@greenearthfoods.com	Basic
3	Modern Homes Realty	Real Estate	Casey Smith	casey.s@modernhomesrealty.com	Custom
*/
Invoking: `sql_db_query` with `SELECT ServicePackage FROM client_info WHERE CompanyName = 'Tech Innovations Ltd.'`


[('Premium',)]The type of service we provide to Tech Innovations is a "Premium" service package.

> Finished chain.


{'input': 'What type of service do we provide to Tech Innovations?',
 'output': 'The type of service we provide to Tech Innovations is a "Premium" service package.'}

## Chaining SQLAgent and Retriever

In [41]:
from langchain.globals import set_debug

set_debug(True)

In [46]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
from langchain_core.prompts import (
    ChatPromptTemplate,

)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 2})


prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever, "context_b": agent_executor,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | llm | output_parser

In [43]:
out = chain.invoke("What are the important points from our last meeting")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What are the important points from our last meeting"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context_a,context_b,question>] Entering Chain run with input:
{
  "input": "What are the important points from our last meeting"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context_a,context_b,question> > 4:chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "What are the important points from our last meeting"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context_a,context_b,question> > 5:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What are the important points from our last meeting"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context_a,context_b,question> > 5:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What are the important points f

In [44]:
print(out)

The important point from the last meeting was an upcoming "Email Newsletter" with the topic "Major Update Announcement!" scheduled by the Marketing Department for May 14, 2024.


In [69]:
chain

{
  context_a: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000186C850F490>, search_kwargs={'k': 6}),
  context_b: AgentExecutor(name='SQL Agent Executor', verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
               agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
             })
             | ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correc

## MultiQuery Retriever

Now, the Multiquery retriever will split the initial query into multiple queries so we can retrieve information for each query, enhancing the retrieving process

In [7]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [48]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(), llm=llm
)

In [8]:
question = "What are the important points from our last meeting"

docs = multiquery_retriever.get_relevant_documents(query=question)
len(docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you summarize the key takeaways from our most recent meeting?', '2. What were the main highlights discussed during our last meeting?', '3. Could you outline the significant points covered in our previous meeting?']


4

In [17]:
docs

[Document(page_content="Client Meeting Minutes: Tech Innovations Ltd.\nMeeting Details\nDate: 2024-04-25\nTime: 10:00 AM - 11:30 AM\nLocation: Digital Marketing Agency Conference Room\nAttendees: Alex Johnson (Tech Innovations Ltd.), Marketing Team (Digital Marketing Agency)\nObjectives of the Meeting\n1. Review the performance of the 'Tech Trendsetters' campaign.\n2. Discuss feedback from Tech Innovations Ltd. on campaign outcomes.\n3. Explore opportunities for upcoming marketing initiatives.\n4. Align on strategies and budget for the next quarter.\nSummary of Discussion\n- The 'Tech Trendsetters' campaign results were presented, highlighting the achievement of all key objectives, including increased website traffic, social media engagement, and lead generation.\n- Alex Johnson expressed satisfaction with the campaign's performance and the positive impact on Tech Innovations Ltd.'s brand visibility.\n- Discussions were held on potential areas for improvement, particularly in expanding

In [21]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate

)


retriever_prompt_str = """You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. Your
    answer can be guided by the given context, but it is not limited to it.
    Provide these alternative questions separated by newlines.
    Context: {context}
    Original question: {question} """
retriever_prompt = ChatPromptTemplate.from_template(retriever_prompt_str)

retrieval_for_retriever = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
)

multiquery_retriever_chain = retrieval_for_retriever | retriever_prompt | llm | output_parser

In [19]:
out = multiquery_retriever_chain.invoke('who are the main clients?')
print(out)

content='1. Which individuals or teams are the primary clients involved in the meetings?\n2. Who are the key attendees representing the client companies in the meeting minutes?\n3. Can you identify the main clients or stakeholders participating in the documented meetings?\n4. Who are the primary client representatives mentioned in the meeting summaries?\n5. Which specific clients or teams are highlighted as significant participants in the meeting discussions?' response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 808, 'total_tokens': 886}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}


In [22]:
retriever.invoke('who is my client')

[Document(page_content="Client Meeting Minutes: Tech Innovations Ltd.\nMeeting Details\nDate: 2024-04-25\nTime: 10:00 AM - 11:30 AM\nLocation: Digital Marketing Agency Conference Room\nAttendees: Alex Johnson (Tech Innovations Ltd.), Marketing Team (Digital Marketing Agency)\nObjectives of the Meeting\n1. Review the performance of the 'Tech Trendsetters' campaign.\n2. Discuss feedback from Tech Innovations Ltd. on campaign outcomes.\n3. Explore opportunities for upcoming marketing initiatives.\n4. Align on strategies and budget for the next quarter.\nSummary of Discussion\n- The 'Tech Trendsetters' campaign results were presented, highlighting the achievement of all key objectives, including increased website traffic, social media engagement, and lead generation.\n- Alex Johnson expressed satisfaction with the campaign's performance and the positive impact on Tech Innovations Ltd.'s brand visibility.\n- Discussions were held on potential areas for improvement, particularly in expanding

In [68]:
#testing the multiquery retriever itself
# I want to work on adding context to this part also
multiquery_prompt_template='''You are an AI language model assistant with a specialization in business intelligence and client management. Your task is to dissect the given user question into five different versions, each focusing on a unique aspect of business understanding or client-specific inquiry. This approach aims to retrieve the most relevant documents from a vector database, overcoming the limitations of distance-based similarity searches by covering various business perspectives and client needs. Consider aspects like financial implications, strategic decisions, client relationship dynamics, and operational efficiency.

Provide these alternative questions separated by newlines.

Original question: {question}
'''


multiquery_prompt_template = PromptTemplate(input_variables=['question'], template=multiquery_prompt_template)


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
test_multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 1}), 
    llm=llm,
    prompt=multiquery_prompt_template,
    parser_key='lines'
)




In [69]:
out = test_multiquery_retriever.invoke({'question':'Give me all i need to know about our client Tech Innovations Ltd. in terms of them, their main goal with the campaigns and spending and posting info about their campaigns'})

[chain/start] [1:retriever:Retriever > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": {
    "question": "Give me all i need to know about our client Tech Innovations Ltd. in terms of them, their main goal with the campaigns and spending and posting info about their campaigns"
  }
}
[llm/start] [1:retriever:Retriever > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an AI language model assistant with a specialization in business intelligence and client management. Your task is to dissect the given user question into five different versions, each focusing on a unique aspect of business understanding or client-specific inquiry. This approach aims to retrieve the most relevant documents from a vector database, overcoming the limitations of distance-based similarity searches by covering various business perspectives and client needs. Consider aspects like financial implications, strategic decisions, client relationshi

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the key details about Tech Innovations Ltd. that we should be aware of in terms of their business operations and market positioning?', "2. Can you provide insights into Tech Innovations Ltd.'s primary objectives with their marketing campaigns and how their spending aligns with these goals?", "3. What specific information can you share about Tech Innovations Ltd.'s posting strategies and the effectiveness of their campaigns in reaching their target audience?", "4. How does Tech Innovations Ltd.'s campaign spending compare to industry benchmarks, and what implications does this have for their overall business performance?", '5. In what ways can we enhance our client relationship with Tech Innovations Ltd. based on their campaign performance data and strategic goals?']


[llm/end] [1:retriever:Retriever > 2:chain:LLMChain > 3:llm:ChatOpenAI] [3.73s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. What are the key details about Tech Innovations Ltd. that we should be aware of in terms of their business operations and market positioning?\n2. Can you provide insights into Tech Innovations Ltd.'s primary objectives with their marketing campaigns and how their spending aligns with these goals?\n3. What specific information can you share about Tech Innovations Ltd.'s posting strategies and the effectiveness of their campaigns in reaching their target audience?\n4. How does Tech Innovations Ltd.'s campaign spending compare to industry benchmarks, and what implications does this have for their overall business performance?\n5. In what ways can we enhance our client relationship with Tech Innovations Ltd. based on their campaign performance data and strategic goals?",
        "generation_info": {
          "finish_reason": "s

In [70]:
out

[Document(page_content='Marketing Strategy Plan\n1. Company Overview\nTech Innovations Ltd. is at the forefront of the technology industry, providing cutting-edge solutions for businesses and consumers alike. With a focus on innovation, quality, and customer service, Tech Innovations aims to lead the market in software development, hardware production, and IT consulting services.\n2. Marketing Objectives\nThe primary marketing objectives for Tech Innovations Ltd. for the upcoming year are to increase brand awareness, enhance online presence, and drive product sales through targeted digital marketing campaigns. Specific goals include:\n- Increase website traffic by 30% within the next 12 months.\n- Grow social media following by 25% across all platforms.\n- Generate 20% more leads through online marketing efforts.\n- Achieve a 15% rise in sales of flagship products.\n3. Target Audience\nOur target audience consists of tech-savvy consumers, ranging from young professionals to industry ex

## Prompt

Lastly, we improve the query to make it better for our specific use case

In [97]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate

)
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

multiquery_prompt_template='''You are an AI language model assistant with a specialization in business intelligence and client management. Your task is to dissect the given user question into five different versions, each focusing on a unique aspect of business understanding or client-specific inquiry. This approach aims to retrieve the most relevant documents from a vector database, overcoming the limitations of distance-based similarity searches by covering various business perspectives and client needs. Consider aspects like financial implications, strategic decisions, client relationship dynamics, and operational efficiency.

Provide these alternative questions separated by newlines.

Original question: {question}
'''


multiquery_prompt_template = PromptTemplate(input_variables=['question'], template=multiquery_prompt_template)


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(), llm=llm, prompt=multiquery_prompt_template
)

#We coudl play with the sql agent prompt but we wont do it here
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)


prompt_str = """As a business intelligence AI, your goal is to synthesize information from multiple sources to provide a comprehensive answer to the main question. Consider the implications of the data from both the Document Insights and Data Insights in the context of our client-focused business environment. Analyze the combined data to offer strategic insights, actionable recommendations, and any relevant data points or trends that support your conclusions.

Ensure your response is concise, data-driven, and directly addresses the business needs related to the question. Avoid generic responses by leveraging the specific information provided.

Context:
- Document Insights: {multiquery}
- Data Insights: {SQLAgent}

Main Question: {question}

Answer (please provide a structured and reasoned response based on the context above):
"""
prompt = ChatPromptTemplate.from_template(prompt_str)

multiquery_retrieval = RunnableParallel(
    {
        "SQLAgent": agent_executor,
        "multiquery": multiquery_retriever,
        "question": RunnablePassthrough()
    }
)



chain_with_multiquery = multiquery_retrieval | prompt | llm 

In [115]:
from IPython.display import Image

Image(chain_with_multiquery.get_graph().draw_png())


ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

In [100]:
out = chain_with_multiquery.invoke('Give me all i need to know about tech innovation in terms of them, their main goal with the campaigns and spending and posting info about their campaigns')

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Give me all i need to know about tech innovation in terms of them, their main goal with the campaigns and spending and posting info about their campaigns"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<SQLAgent,multiquery,question>] Entering Chain run with input:
{
  "input": "Give me all i need to know about tech innovation in terms of them, their main goal with the campaigns and spending and posting info about their campaigns"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<SQLAgent,multiquery,question> > 3:chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "Give me all i need to know about tech innovation in terms of them, their main goal with the campaigns and spending and posting info about their campaigns"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<SQLAgent,multiquery,question> > 4:retriever:Retriever > 5:cha

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the financial implications of tech innovation for companies, including their campaign goals and spending patterns?', "2. How do strategic decisions around tech innovation impact companies' campaign strategies and spending habits?", "3. What insights can be gained from analyzing tech companies' posting information in relation to their innovation campaigns?", '4. How do tech companies manage client relationships through their innovative campaigns and spending strategies?', "5. In what ways can operational efficiency be improved by understanding tech companies' approaches to innovation campaigns and spending?"]


[llm/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<SQLAgent,multiquery,question> > 4:retriever:Retriever > 5:chain:LLMChain > 6:llm:ChatOpenAI] [2.10s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1. What are the financial implications of tech innovation for companies, including their campaign goals and spending patterns?\n2. How do strategic decisions around tech innovation impact companies' campaign strategies and spending habits?\n3. What insights can be gained from analyzing tech companies' posting information in relation to their innovation campaigns?\n4. How do tech companies manage client relationships through their innovative campaigns and spending strategies?\n5. In what ways can operational efficiency be improved by understanding tech companies' approaches to innovation campaigns and spending?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",

In [103]:
out

AIMessage(content="Based on the combined insights from the Document and Data sources, here is a comprehensive overview of Tech Innovations Ltd. and their digital marketing campaigns:\n\n1. Tech Innovations Ltd. Overview:\n   - Tech Innovations Ltd. is a leading technology company focusing on innovation, quality, and customer service.\n   - Their primary marketing objectives include increasing brand awareness, enhancing online presence, and driving product sales through targeted digital marketing campaigns.\n   - The target audience consists of tech-savvy consumers and small to medium-sized businesses in need of technology solutions.\n\n2. Digital Marketing Campaigns:\n   - Tech Innovations Ltd. has run multiple campaigns with varying objectives and budgets:\n     - CampaignID: C001 (Tech Trendsetters) - Objective: Brand Awareness, Budget: $5000\n     - CampaignID: C002 (Organic Reach Boost) - Objective: Lead Generation, Budget: $3000\n     - CampaignID: C003 (Luxury Living Launch) - Ob

In [86]:
print(out)

Based on the combined insights from the documents and data provided, here is a comprehensive overview of Tech Innovations Ltd. in terms of their digital marketing campaigns, main goals, and spending:

1. Tech Innovations Ltd. Overview:
   - Tech Innovations Ltd. is a leading technology company focusing on software development, hardware production, and IT consulting services.
   - Their primary marketing objectives include increasing brand awareness, enhancing online presence, driving product sales, and targeting tech-savvy consumers and businesses.

2. Campaign Performance:
   - The 'Tech Trendsetters' campaign aimed to increase brand awareness and showcase Tech Innovations Ltd. as a leader in cutting-edge technology solutions.
   - Key objectives included increasing website traffic by 20%, growing social media engagement by 30%, generating 500 new leads, and enhancing brand recognition within the technology industry.
   - The campaign exceeded expectations with a 25% increase in websi